In [15]:
import os
print(os.getcwd())

/home/jupyter/project/yandex_metrika_cloud_case


In [16]:
import pandas as pd
import requests

# Загрузка данных в `ClickHouse`


In [17]:
hits_df = pd.read_csv('metrika_cloud_case_data_hits.csv', sep = '\t')
visits_df = pd.read_csv('metrika_cloud_case_data_visits.csv', sep = '\t')

## Подключение и настройка
https://cloud.yandex.ru/docs/managed-clickhouse/
(см. слайды)

## Данные для доступа
* Из интерфейса облака в разделе хосты копируем имя хост в переменную `CH_HOST_NAME`
* Используем заведенного юзера в переменной `CH_USER`
* Используем заведенного юзера в переменной `CH_USER`

* Сохраним пароль заведенного пользователя в текстовый файл `.chpass.txt`
* В переменную `CH_PASS` считаем содержимое файла `.chpass.txt`
* В переменную `cacert` поместим путь к сертификату для подключения к серверу. Файл `YandexInternalRootCA.crt` должен лежать в репозитории

In [18]:
#----------Вводить свои данные в нижние 3 переменные--------------
CH_HOST_NAME = 'rc1b-vsrkuug8qh3pkkeg.mdb.yandexcloud.net'
CH_USER      = 'User_main'
CH_DB_NAME   = 'ym_sanok'

#-------------------------------------------
CH_PASS      = open('../.chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'YandexInternalRootCA.crt'

##  Функции для интеграции с ClickHouse

В файле `some_funcs` есть класс `simple_ch_client` для работы с ClickHouse

Сначала надо создать экземпляр класса, инициализировав его начальными параметрами - хост, пользователь, пароль и путь к сертификату
`simple_ch_client(CH_HOST, CH_USER, CH_PASS, cacert)`

В классе есть 4 метода:
* `.get_version()` - получает текущую версию ClickHouse. Хороший способ проверить, что указанные при инициализации параметры работают
* `.get_clickhouse_data(query)` - выполняет запрос `query` и возвращает результат в текстовом формате
* `.get_clickhouse_df(query)` - выполняет запрос `query` и возвращает результат в виде DataFrame
* `.upload(table, content)` - загружает таблицу `content`, которая подается в текстовом формате в таблицу ClickHouse'а с именем `table`


## Проверяем ClickHouse
Используя заготовленные выше переменные, проверим доступ до сервера (как в документации https://cloud.yandex.ru/docs/managed-clickhouse/operations/connect#connection-string)
Этот метод реализован в методе `.get_version()` класса для работы с ClickHouse
При успешном подключении не произойдет никакой ошибки при выполнении этого метода, и он сам вернет версию сервера ClickHouse (например `21.3.2.5`)

In [19]:
import some_funcs
from some_funcs import simple_ch_client

In [20]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)

In [21]:
my_client.get_version()

25.8.14.17-yc.0



### Загружаем данные

#### Хиты

In [22]:
hits_df.head()

,ym:pv:browser,ym:pv:clientID,ym:pv:date,ym:pv:dateTime,ym:pv:deviceCategory,ym:pv:lastTrafficSource,ym:pv:operatingSystemRoot,ym:pv:URL
0,samsung_internet,1706592217203130976,2026-02-08,2026-02-08 21:43:11,2,direct,android,https://sanok.ru/mebel-dlya-vannoi/zerkal-nye-...
1,chromemobile,1769362260534895026,2026-02-08,2026-02-08 23:58:29,2,direct,android,https://sanok.ru/unitazi/podvesnie-unitazi-s-i...
2,samsung_internet,1770584320715078576,2026-02-08,2026-02-08 23:59:43,2,internal,android,https://sanok.ru/dushevye-kabiny/dk/otkryvanie...
3,yandex_browser,1768560319418918246,2026-01-18,2026-01-18 23:29:12,2,direct,ios_double,https://sanok.ru/santehnika-dlja-invalidov/van...
4,opera,1768468324564534228,2026-02-01,2026-02-01 23:44:58,1,organic,windows,https://sanok.ru/santehnika-iz-nerjaveiki/pite...


In [23]:
hits_df.rename(columns={'ym:pv:browser':'Browser',
                'ym:pv:clientID':'ClientID',
                'ym:pv:date':'EventDate',
                'ym:pv:dateTime':'EventTime',
                'ym:pv:deviceCategory':'DeviceCategory',
                'ym:pv:lastTrafficSource':'TraficSource',
                'ym:pv:operatingSystemRoot':'OSRoot',
                'ym:pv:URL':'URL'}, inplace = True)

In [24]:
q = f'drop table if exists {CH_DB_NAME}.hits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.hits (
    Browser String,
    ClientID UInt64,
    EventDate Date,
    EventTime DateTime,
    DeviceCategory String,
    TraficSource String,
    OSRoot String,
    URL String
) ENGINE = MergeTree() ORDER BY (intHash32(ClientID), EventDate) SAMPLE BY intHash32(ClientID) SETTINGS index_granularity=8192
'''

my_client.get_clickhouse_data(q)

''

In [25]:
my_client.upload(
    f'{CH_DB_NAME}.hits',
    hits_df.to_csv(index = False, sep = '\t'))

''

#### Визиты

In [26]:
visits_df.head()

,ym:s:browser,ym:s:clientID,ym:s:date,ym:s:dateTime,ym:s:deviceCategory,ym:s:lastTrafficSource,ym:s:operatingSystemRoot,ym:s:purchaseID,ym:s:purchaseRevenue,ym:s:startURL
0,yandexbrowserlite,1768685000224899485,2026-01-18,2026-01-18 00:23:21,2,organic,android,[],[],https://sanok.ru/vanny/stalnye/razmery_150x70/
1,yandex_browser,1769252766367586565,2026-01-24,2026-01-24 14:07:35,1,organic,windows,[],[],https://sanok.ru/mebel-dlya-vannoi/tumby/tumba...
2,chrome,1768319064528192523,2026-01-13,2026-01-13 18:44:23,1,organic,windows,[],[],https://sanok.ru/aksessuary-dlya-obschestvenny...
3,safari_mobile,1768196071907883357,2026-01-12,2026-01-12 08:34:31,2,organic,ios_double,[],[],https://sanok.ru/mojki/mojki-kuhonnie-iz-iskus...
4,chrome,1769780812115280481,2026-01-30,2026-01-30 16:46:51,1,organic,windows,[],[],https://sanok.ru/index.php?route=checkout/simp...


In [27]:
visits_df.rename(columns={'ym:s:browser':'Browser',
                'ym:s:clientID':'ClientID',
                'ym:s:date':'StartDate',
                'ym:s:dateTime':'StartTime',
                'ym:s:deviceCategory':'DeviceCategory',
                'ym:s:lastTrafficSource':'TraficSource',
                'ym:s:operatingSystemRoot':'OSRoot',
                'ym:s:purchaseRevenue': 'Purchase.Revenue', 
                'ym:s:purchaseID': 'Purchase.ID',
                'ym:s:startURL':'StartURL'}, inplace = True)

In [29]:
visits_df['Purchases'] = visits_df['Purchase.Revenue'].map(lambda x: x.count(',') + 1 if x != '[]' else 0 )
visits_df['Revenue'] = visits_df['Purchase.Revenue'].map(lambda x: sum(map(int,x[1:-1].split(','))) if x != '[]' else 0)

ValueError: invalid literal for int() with base 10: '28145.0'

In [30]:
visits_df['Purchases'] = visits_df['Purchase.Revenue'].map(lambda x: x.count(',') + 1 if x != '[]' else 0)
visits_df['Revenue'] = visits_df['Purchase.Revenue'].map(lambda x: sum(map(float, x[1:-1].split(','))) if x != '[]' else 0)

In [31]:
visits_df.drop(columns=['Purchase.ID','Purchase.Revenue'], inplace=True)

In [32]:
q = f'drop table if exists {CH_DB_NAME}.visits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.visits (
    Browser String,
    ClientID UInt64,
    StartDate Date,
    StartTime DateTime,
    DeviceCategory UInt8,
    TraficSource String,
    OSRoot String,
    Purchases Int32,
    Revenue Double,
    StartURL String
) ENGINE = MergeTree() ORDER BY (intHash32(ClientID), StartDate) SAMPLE BY intHash32(ClientID) SETTINGS index_granularity=8192
'''

my_client.get_clickhouse_data(q)

''

In [33]:
my_client.upload(
    f'{CH_DB_NAME}.visits',
    visits_df.to_csv(sep='\t', index =False))

''